### Neutral reviews (3 stars)

In [1]:
%%capture
!pip install -r ../requirements.txt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import fasttext
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.cluster.util import cosine_distance
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from wordcloud import WordCloud
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import model_selection, svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df_raw = pd.read_csv('../data/raw/Big_AHR.csv')

In [4]:
df_3_stars = df_raw[df_raw['rating']==3]

In [15]:
for text in df_3_stars['review_text'].sample(5).tolist():
    print(text)
    print()

Está bien situado, aunque las calles de alrededor son muy sucias...por la noche se oía ruido de la calle. Tuvimos bichos en la habitación, aunque el personal fumigó y nos ofreció el cambio la situación fue algo desagradable. Aun así, tiene buen precio y buen desayuno (merece la pena cogerlo).

La ubicación es perfecta, entra el acceso a la Alcazaba y la Catedral. Puedes recorrer todo el centro e ir a la playa de la Malagueta y la zona del MuelleUno andando. El parking está a escasos metros y el acceso al mismo y las plazas son correctos. La habitación daba a un patio interior y era tranquila. El hotel está algo antiguo, pero cumple para una estancia de ciudad. Alrededor hay muchos establecimientos para desayunar, comer y cenar.

La posada Doña Lupe se puede ver de muchas maneras. No es un lugar para estarse en las habitaciones, ya que éstas sólo sirven para dormir y gracias. Está poco cuidado (faltaban muchas bombetas, por ejemplo) y se podría decir que la limpieza era la justa. Aún as

In this sample of neutral reviews, it can be seen that the comments highlight both positive and negative aspects.
Therefore, each comment will be separated into sentences, with the aim of being classified as positive or negative.

The division of each comment into sentences and the text processing was done *using create_df_sentences.py*

In [10]:
df_sentences_3_stars = pd.read_csv('../data/interim/sentences_3_stars.csv')

df_sentences_1_star = pd.read_csv('../data/interim/sentences_1_star.csv')
df_sentences_5_stars = pd.read_csv('../data/interim/sentences_5_stars.csv')

In [11]:
df_sentences_5_stars = df_sentences_5_stars.sample(len(df_sentences_1_star))

In [14]:
print('Examples of negative sentences:')
df_sentences_1_star.sample(3)

Examples of negative sentences:


,text,text_processed
11222,"Es, con diferencia, el peor hotel en el que no...",diferencia peor albergado mujer cadena nh sido...
546,…,NaN
5248,No lo recomendamos,no recomendamos


In [30]:
print('Examples of positive sentences:')
df_sentences_5_stars.sample(3)

Examples of positive sentences:


,text,text_processed
26391,Una decoración muy chula y el servicio y atenc...,decoracion chula servicio atencion impecables
23917,Habitación confortable para su precio,habitacion confortable precio
47929,Totalmente recomendable!!,totalmente recomendable


In [12]:
# Add column to indicate sentiment
df_sentences_1_star['label'] = 0
df_sentences_5_stars['label'] = 1

In [13]:
# Combine positive and negative sentences into one df
df_sentences_labeled = pd.concat([df_sentences_1_star, df_sentences_5_stars]).reset_index(drop=True)

In [14]:
df_sentences_labeled['label'] = df_sentences_labeled['label'].astype('category')

##### Construction of the model for sentences classification

In [15]:
X = df_sentences_labeled[['text_processed']]
y = df_sentences_labeled['label']

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=12)

In [17]:
model = Pipeline([
    ('count_vectorizer', CountVectorizer(max_features=200, ngram_range=(1,2), lowercase=False)),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier', MultinomialNB())     # SVC(kernel='rbf', probability=True)
])

In [18]:
model.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1, 17827]

In [38]:
model2 = Pipeline([
    ('count_vectorizer', CountVectorizer(max_features=200, ngram_range=(1,2), lowercase=False))
])

In [39]:
model2.fit(X_train, y_train)

Pipeline(steps=[('count_vectorizer',
                 CountVectorizer(lowercase=False, max_features=200,
                                 ngram_range=(1, 2)))])